#### Variance Reduction by Control Variates. 

For the control variates technique an accurate estimate of the value of an option that is similar to the one that you would like to price is required. 

For valuation of an Asian option based on arithmetic averages one can use the value of an Asian option based on geometric averages. 

This case can be solved analytically.

In [18]:
#Imports

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm,gmean
np.random.seed(42)

Φ = norm.cdf



#### Theoretical stuff:

***Black-Scholes*** model describes the evolution of a stock price in the risk-neutral world through the stochastic differential equation (SDE)

$$dS(t) = rS(t)dt + \sigma S(t)dW (t) $$

The solution of the equation above is:
$$ S(T)=S(0)\exp{\big [r−\frac{1}{2}\sigma]T+\sigma \sqrt{TZ} \big)} ,$$

The payoff of an **Asian call** option based on geometric averages is given by:
$$\big(\tilde{A}_N - K \big)^{+} $$

Where $$\tilde{A}_N = \Big( \prod_{i=0}^{N}S \big(\frac{iT}{N} \big) \Big)^{\frac{1}{N+1}} $$

(Some long derivations that I skipped....)


Hence, the price $C_{g}^{A}(S(0), T )$ can be obtained by the risk-neutral method

$$ C_{g}^{A}(S(0), T ) = \exp{(−rT)}\big( S(0)\exp{(\tilde{r}T)}\Phi(\tilde{d}_1)−K\Phi\tilde{d}_1\big)$$

Where $$ \tilde{r} = \frac{[ r − \frac{1}{2} \sigma^2 ] + \tilde{\sigma}^2}{2} $$

And $$ \tilde{d}_1 = \frac{\log{\frac{S(0)}{K}} + (\tilde{r} + 0.5 \tilde{\sigma}^2)T}{\sqrt{T}\tilde{\sigma}}$$
$$ \tilde{d}_2 = \frac{\log{\frac{S(0)}{K}} + (\tilde{r} - 0.5 \tilde{\sigma}^2)T}{\sqrt{T}\tilde{\sigma}}$$

In [19]:

def geometric_mean(iterable):
    xs = np.array(iterable)
    
    return np.exp(np.sum(np.log(xs))/xs.size)

def bs_d1(t, T, St, K, r, σ):
    τ = T - t
    log_term = np.log(St/K)
    σ_term = (r + 0.5 * σ**2)*τ
    numerator = log_term + σ_term
    denominator = σ*np.sqrt(τ)
    return numerator / denominator

def bs_d2(t, T, St, K, r, σ):
    τ = T - t
    return bs_d1(t, T, St, K, r, σ) - σ*np.sqrt(τ)

def euro_call_valuation(t, T, St, K, r, σ):
    τ = T - t
    d1 = bs_d1(t, T, St, K, r, σ)
    d2 = bs_d2(t, T, St, K, r, σ)
    return St * Φ(d1) - np.exp(-r * τ) * K * Φ(d2)

def asian_gm_valuation(S0, T, K, r, σ, N):
    σa = σ * np.sqrt((2 * N + 1) / (6 * (N + 1)))
    ra = 0.5 * ((r - 0.5 * σ**2) + σa**2)
    return np.exp((ra - r) * T) * euro_call_valuation(0, T, S0, K, r, σ)

def vanilla_call_payoff(S, K):
    return max(S - K, 0)

def asian_gm_call_payoff(Sts, K):
    Ag = geometric_mean(Sts)
    return vanilla_call_payoff(Ag, K)
    

In [20]:
def fst(xs):
    return xs[0]

def snd(xs):
    return xs[1]

def lst(xs):
    return xs(-1)

In [29]:
np.random.seed(42)
def euler_GBM(S0, T, M, r, σ):
    δt = T / M
    Sts = np.empty(M+1)
    Sts[0] = S0
    for m in range(M):
        zm = np.random.normal(loc=0.0, scale=1.0)
        Sts[m+1] = Sts[m] * (1.0 + r * δt + σ * np.sqrt(δt) * zm)
    return Sts

def euler_GBM_ensemble(S0, T, M, r, σ, nsim):
    Stss = np.empty((nsim, M+1))
    for i in range(nsim):
        Stss[i] = euler_GBM(S0, T, M, r, σ)
    return Stss

def euler_GBM_asian_gm_payoffs(S0, T, M, r, σ, K, nsim):
    Stss = euler_GBM_ensemble(S0, T, M, r, σ, nsim)
    # payoffs = [1]*nsim
   
    payoffs = np.empty(nsim)
    for i,Sts in enumerate(Stss):
        payoffs[i] = asian_gm_call_payoff(Sts, K)
    # for i in range(nsim):
    #     payoffs[i] = asian_gm_call_payoff(Stss[i][0], K)
    return payoffs

def opt_value_empirical(payoffs, r, T):
    payoff_expectation = np.mean(payoffs)
    payoff_std = np.std(payoffs)
    discount = np.exp(-r * T)
    opt_value_estimate = payoff_expectation * discount
    opt_value_estimate_std = payoff_std * discount
    return opt_value_estimate, opt_value_estimate_std
    
def mc_asian_gm_valuation(S0, T, M, K, r, σ, nsim):
    payoffs = euler_GBM_asian_gm_payoffs(S0, T, M, r, σ, K, nsim)

    return opt_value_empirical(payoffs, r, T)

In [36]:
nsim = 1000  # number of simulations
S0 = 100.0  # initial stock price
K = 99.0  # strike price
σ = 0.2  # volatility
r = 0.06  # risk-free interest rate
T = 1.0  # time till maturity
M = 365  # number of observations

# theoretical call option value
asian_gm_valuation(S0, T, K, r, σ, M)

11.165712012250578

In [23]:
var = euler_GBM_ensemble(S0, T, M, r, σ, nsim)


In [43]:

result = mc_asian_gm_valuation(S0, T, M, K, r, σ, nsim) 
print(result)


(6.517366991843357, 8.11238914990104)
